In [90]:
# import os
import json
import time
import boto3
import pandas as pd
# from imblearn.over_sampling import SMOTE
# from sklearn.model_selection import train_test_split

import sagemaker
# from sagemaker.inputs import TrainingInput
# from sagemaker.xgboost.estimator import XGBoost
# from sagemaker.debugger import Rule, rule_configs
# from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

# from sagemaker import clarify
# from sagemaker.session import Session


In [2]:
# Set region, boto3 and SageMaker SDK variables¶

#You can change this to a region of your choice
region = sagemaker.Session().boto_region_name
print("Using AWS Region: {}".format(region))

boto3.setup_default_session(region_name=region)
boto_session = boto3.Session(region_name=region)

s3_client = boto3.client('s3', region_name=region)
sagemaker_boto_client = boto_session.client('sagemaker')

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_boto_client)

sagemaker_role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity()["Account"]

random_state = 42

Using AWS Region: us-east-1


In [26]:
%store -r
%store

Stored variables and their in-db values:
clarify_data_uri                    -> 's3://sagemaker-us-east-1-367158743199/sagemaker-t
data_prefix                         -> 'sagemaker-tutorial/data'
default_bucket                      -> 'sagemaker-us-east-1-367158743199'
endpoint_config_name                -> 'sagemaker-tutorial-xgboost-smote-endpoint-config'
endpoint_name                       -> 'sagemaker-tutorial-xgboost-smote-endpoint'
feature_group_name                  -> 'FG-flow-sm-tutorial-31-16-16-17-9f41d66b'
header                              -> ['LABEL', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIA
hyperparameters                     -> {'max_depth': '5', 'eta': '0.2', 'gamma': '4', 'mi
local_processed_data                -> '../sm-tutorial/02_build_train/processed_data'
model_2_name                        -> 'sagemaker-tutorial-xgboost-smote'
model_data                          -> 's3://sagemaker-us-east-1-367158743199/tf2-resnet-
mp2_arn                             -> 'arn:a

# Approve the Second model

In [4]:
second_model_package = sagemaker_boto_client.list_model_packages(ModelPackageGroupName=mpg_name)['ModelPackageSummaryList'][0]
model_package_update = {
    'ModelPackageArn': second_model_package['ModelPackageArn'],
    'ModelApprovalStatus': 'Approved'
}

update_response = sagemaker_boto_client.update_model_package(**model_package_update)

# Create an endpoint config + endpoint

In [ ]:
primary_container

In [6]:
approved_model_name = model_2_name

endpoint_name = f'{approved_model_name}-endpoint'
endpoint_instance_count = 1
endpoint_instance_type = "ml.m4.xlarge"


primary_container = {'ModelPackageName': second_model_package['ModelPackageArn']}
endpoint_config_name=f'{approved_model_name}-endpoint-config'
existing_configs = len(sagemaker_boto_client.list_endpoint_configs(NameContains=endpoint_config_name, MaxResults = 30)['EndpointConfigs'])

if existing_configs == 0:
    create_ep_config_response = sagemaker_boto_client.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[{
            'InstanceType': endpoint_instance_type,
            'InitialVariantWeight': 1,
            'InitialInstanceCount': endpoint_instance_count,
            'ModelName': approved_model_name,
            'VariantName': 'AllTraffic'
        }]
    )
    %store endpoint_config_name
    %approved_model_name

In [10]:
existing_endpoints = sagemaker_boto_client.list_endpoints(NameContains=endpoint_name, MaxResults = 30)['Endpoints']
if not existing_endpoints:
    create_endpoint_response = sagemaker_boto_client.create_endpoint(
        EndpointName=endpoint_name,
        EndpointConfigName=endpoint_config_name)
    %store endpoint_name

endpoint_info = sagemaker_boto_client.describe_endpoint(EndpointName=endpoint_name)
endpoint_status = endpoint_info['EndpointStatus']

while endpoint_status == 'Creating':
    endpoint_info = sagemaker_boto_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_status = endpoint_info['EndpointStatus']
    print('Endpoint status:', endpoint_status)
    if endpoint_status == 'Creating':
        time.sleep(60)



Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: InService


## Create a predictor

In [11]:
predictor = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session)

# Make predictions!

### sample a row from the test data

In [14]:
!aws s3 cp $test_data_uri ./data/

download: s3://sagemaker-us-east-1-367158743199/sagemaker-tutorial/data/test.csv to data/test.csv


In [39]:
df_test = pd.read_csv('./data/test.csv', header=None)
df_test.columns = header

### get sample data. from online featre store

In [40]:
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = sagemaker.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_boto_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

In [43]:
fg_response = featurestore_runtime.get_record(
    FeatureGroupName=feature_group_name, 
    RecordIdentifierValueAsString= str(1.0)

)

if (fg_response.get('Record')):
    fg_response = fg_response['Record']
    df_sample = pd.DataFrame(fg_response).set_index('FeatureName')
else:
    print ("No Record returned / Record Key  \n")


In [63]:
df_test = pd.DataFrame(fg_response).set_index('FeatureName').T.reset_index()

df_test = df_test.drop(columns=['index', 'LABEL', 'ID', 'EVENT_TIME'])

test_input = ','.join(df_test.values.flatten().tolist())

In [95]:
results = predictor.predict(test_input, initial_args = {"ContentType": "text/csv"})
prediction = json.loads(results)
prediction
print (f'Probablitity of default is:', prediction)

Probablitity of default is: 0.7100669741630554
